# Setup

First, we need to initialize the object (a Pertpy package internal dataset), and if the object isn't already preprocessed and/or clustered, perform those steps.

## Imports & Options

In [ ]:
%load_ext autoreload
%autoreload 2

import  as cr
from  import Crispr
import pertpy as pt
import scanpy as sc
import pandas as pd
import numpy as np

# Options
print("\nAnalysis Functions\n\t" + "\n\t".join(list(pd.Series(
    [np.nan if "__" in x else x for x in dir(cr.ax)]).dropna())))
file = "perturb-seq"
pd.options.display.max_columns = 100

#  Set Arguments
kwargs_init = dict(assay=None,
                   col_gene_symbols="gene_symbol",
                   col_cell_type="celltype",
                   col_perturbed="perturbation",
                   col_guide_rna="perturbation",
                   col_num_umis=None,
                   kws_process_guide_rna=None,
                   col_condition="target_gene_name",
                   key_control="Control",
                   key_treatment="KO")
file_path = pt.dt.papalexi_2021()

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.


Analysis Functions
	analyze_composition
	cluster
	clustering
	composition
	compute_distance
	find_marker_genes
	perform_augur
	perform_celltypist
	perform_differential_prioritization
	perform_gsea
	perform_mixscape
	perturbations


Output()

## Object

This code instantiates the CRISPR object, which is the main way of interacting with this package as an end-user.

This is more code than you would need in real life; it just ensures that certain public datasets are loaded from the source for various reasons.

In [ ]:
self = Crispr(file_path, **kwargs_init)

# Processing

We will just use defaults for the preprocessing arguments (very little filtering, consistent with the Pertpy tutorial for this object).

## Preprocess

In [ ]:
self.preprocess(kws_hvg=True)  # preprocessing

## Cluster

In [ ]:
_ = self.cluster()

### CellTypist Annotations

Now let's detect cell types using `CellTypist`.

In [ ]:
_ = self.annotate_clusters("ImmuneAllHigh.pkl")

# Inspection

First, you can easily print the `.adata` representation, the gene expression modality `.obs` preview, and columns and keys stored in the object's attributes using the `print()` method.

In [ ]:
self.print()

## Set Up Arguments for Later


This code looks more complicated than it actually would actually be for an end user because it was made to be generalizable across several datasets with particular column names, sizes that make it necessary to subset them in order to run the vignettes in a reasonable period of time, etc.

Basically, you won't need this code as an end user; this is just to choose random subsets of genes and perturbations, etc. that are available in a given example dataset.

In real use cases, you will know what genes and conditions are of interest, and you can manually specify them by simply stating them in the appropriate arguments (such as `target_gene_idents`) or (in many cases) by not specifying the argument (resulting in the code using all available genes, etc.).

In [ ]:
genes = self.rna.var.reset_index()[self._columns["col_gene_symbols"]]
genes_subset = list(pd.Series(genes).sample(10))
target_gene_idents = list(self.rna.obs[self._columns[
    "col_target_genes"]].sample(10))  # 10 random guide gene targets

## Explore Data Descriptives

You can plot and explore descriptives using the `describe()` method.

In [ ]:
_ = self.describe()  # simple
# _ = self.describe(group_by=self._columns["col_target_genes"], plot=True)

# Plots

Create plots applicable to scRNA-seq data broadly (e.g., UMAP, dotplots) without having to do any perturbation-specific analyses.

## Basic Usage

You can create simple plots easily without having to remember a bunch of arguments to specify! 

The most useful is the `genes` argument, which allows you to subset the number of features plotted (useful for spead and layout/interpretability of plots).

In [ ]:
# figs = self.plot(genes=["gene A", "gene B"...])  # to specify specific genes
figs = self.plot(genes=24)  # to specify random subset of this # of genes

# Analyses

The following examples concern CRISPR or other perturbation design-specific analyses.

In [ ]:
# Choose Subset of Target Genes (optional)
tgis = list(pd.Series(target_gene_idents).sample(3)) if len(
    target_gene_idents) > 3 else target_gene_idents  # smaller subset = faster
cct = "majority_voting" if "majority_voting" in self.rna.obs else None

## Augur: Perturbation Responses by Cell Type

**Which cell types are most affected by perturbations?** Quantify perturbation responses by cell type with Augur, which uses supervised machine learning classification of experimental condition labels (e.g., treated versus untreated). The more separable the condition among cells of a given type, the higher the perturbation effect score.

<u> __Features__ </u>  

- Quantify and visualize degree of perturbation response by cell type
- Identify the most important features (genes).

<u> __Input__ </u>  

There are no required arguments. 
* If you want to override defaults drawn from `._columns` and/or `._keys`, specify the appropriate argument (e.g., `col_cell_type`). 
* You can also specify a different `classifer` (default "random_forest_classifer") used in the machine learning classification procedure used to calculate the AUCs/accuracy. 
* You may pass keyword arguments to the Augur predict method by specifying a dictionary in `kws_augur_predict`.
* Specify `select_variance_features` as True to run the original Augur implementation, which removes genes that don't vary much across cell type. If False, use features selected by `scanpy.pp.highly_variable_genes()`, which is faster and sensitively recovers effects; however, the feature selection may yield inflated Augur scores because this reduced feature set is used in training, resulting in it taking advantage of the pre-existing power of this feature selection to separate cell types.
* Specify `n_folds` and/or `subsample_size` to choose the number and sample size of folds in cross-validation.
* Set an integer for `seed` to allow reproducibility across runs.

<u> __Output__ </u>  

Tuple, where the first element is the AnnData object created by the function, the second, the results dictionary, and the third, a dictionary of figures visualizing results. If copy is False (default), these outputs can also be found in `.results["augur"]` and `.figures["augur"]`.

<u> __Notes__ </u>  

- Sub-sample sizes equal across conditions; does not account for perturbation-induced compositional shifts (cell type abundance)
- Scores are for cell types (aggregated across cells, not individual cells)
- Two modes
    - If select_variance_feature=True, 
    - If False, you also have to be sure that "highly_variable_features" is a variable in your data. This can be complicated if you have a separate layer for perturbation data.

In [ ]:
cct = "majority_voting" if "majority_voting" in self.rna.obs else \
    self._columns["col_cell_type"]
_ = self.run_augur(
    col_cell_type=cct,
    # ^ will be label in self._columns by default, but can override here
    col_perturbed=self._columns["col_perturbed"],
    # ^ will be this by default if unspecified, but can override here
    key_treatment=self._keys["key_treatment"],
    # ^ will be this by default if unspecified, but can o verride here
    select_variance_features=True,  # filter by highly variable genes
    classifier="random_forest_classifier", n_folds=3, augur_mode="default",
    kws_umap=kws_umap, subsample_size=5, kws_augur_predict=dict(span=0.7))